# 481n Project Advanced Solution

In [14]:
import cayde
from cayde.well.nlpwell import NLPWell
from cayde.classifier.dtree import DecisionTreeClassifier
from cayde.classifier.ada import AdaBoosterClassifier
from cayde.classifier.svm import SVMClassifier

In [25]:
well = NLPWell("training_data.csv")
well.fetch()

well.input_cols = ["head", "body"]
well.text_cols = ["head"]
well.output_col = "Stance"

In [26]:
well.input_df

,head,body
0,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...
1,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's..."
2,'Nasa Confirms Earth Will Experience 6 Days of...,Thousands of people have been duped by a fake ...
3,Banksy 'Arrested & Real Identity Revealed' Is ...,If you’ve seen a story floating around on your...
4,Woman detained in Lebanon is not al-Baghdadi's...,An Iraqi official denied that a woman detained...
...,...,...
4513,Tropical spider burrows under man's skin throu...,A trip to Bali has turned Dylan Thomas into a ...
4514,"Tour, agent: Tiger Woods not banned","PALM BEACH GARDENS, Fla. -- A journeyman profe..."
4515,Meteorite leaves crater in Nicaraguan capital ...,A blast near the Nicaraguan capital city of Ma...
4516,ESPN to save NFL's image with all-male domesti...,Tonight — finally! — ESPN is going to have an ...


In [27]:
# Work with a smaller dataset in a non-google TPU environment
well = well.getToySample(maxRows=2)

In [28]:
well.df

,Stance,head,body
853,1,Christian Bale will not play Steve Jobs,Though Christian Bale was in final talks to pl...
3679,0,Israeli-Canadian Woman 'Captured' By ISIS Says...,BREAKING: Gill Rosenberg (31) from Tel Aviv fo...


# BERT Experimentation

In [29]:
well.createBERTEncodings()

KeyboardInterrupt: 

In [ ]:
import bert
import tensorflow as tf
import tensorflow_hub as hub
def create_tokenizer_from_hub_module(bert_model_hub: str) -> bert.tokenization.FullTokenizer:
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(bert_model_hub)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
              vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"], tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1")

# API Demonstration

In [6]:
# Returns a list of new columns available for usage
# This creates columns where the text is stemmed and cleaned
well.clean_text()

['head_1gram', 'head_clean', 'body_1gram', 'body_clean']

In [7]:
well.input_cols = ['head', 'body', 'head_clean', 'body_1gram']

In [8]:
well.df.head()

,Stance,head,body,head_clean,body_1gram
2867,1,KC hospital: Patient does not have symptom pro...,"TOPEKA, Kan. (WIBW)- A Kansas City patient is ...",kc hospit patient doe symptom profil ebola,"[topeka, kan, wibw, kansa, citi, patient, trea..."
2094,1,Macaulay Culkin Victim of Another Online Death...,Macaulay Culkin is NOT dead. A “breaking news”...,macaulay culkin victim anoth onlin death hoax,"[macaulay, culkin, dead, break, news, stori, c..."
2198,1,Bugatti Veyron Vandalized With Penis Graffiti?...,This picture of a penis spraypainted on a £1.5...,bugatti veyron vandal peni graffiti viral phot...,"[pictur, peni, spraypaint, million, bugatti, v..."
2520,0,KFC Marijuana Sales To Begin In Colorado? It’s...,KFC (NYSE:YUM) Gets Occupational Business Lice...,kfc marijuana sale begin colorado unbeliev liter,"[kfc, nyse, yum, get, occup, busi, licens, sel..."
1056,1,"Teen Made $72 Million Trading Stocks, Confirms...","If we've learned anything in 2014, it's that t...",teen made 72 million trade stock confirm teen ...,"[learn, anyth, 2014, teenag, fear, rever, prof..."


In [9]:
# This creates columns where the text is converted to bigrams/trigrams
well.createNgrams()

['head_2gram',
 'head_2gram_unique',
 'head_2gram_size',
 'head_2gram_unique_ratio',
 'head_3gram',
 'head_3gram_unique',
 'head_3gram_size',
 'head_3gram_unique_ratio',
 'head_4gram',
 'head_4gram_unique',
 'head_4gram_size',
 'head_4gram_unique_ratio',
 'body_2gram',
 'body_2gram_unique',
 'body_2gram_size',
 'body_2gram_unique_ratio',
 'body_3gram',
 'body_3gram_unique',
 'body_3gram_size',
 'body_3gram_unique_ratio',
 'body_4gram',
 'body_4gram_unique',
 'body_4gram_size',
 'body_4gram_unique_ratio']

In [10]:
# Does the word exist in each text_column?
well.createCountWordFeatures([
    'fake',
    'fraud',
    'hoax',
    'false',
    'deny', 'denies',
    'refute',
    'not',
    'despite',
    'nope',
    'doubt', 'doubts',
    'bogus',
    'debunk',
    'pranks',
    'retract'
])

['head_contains_fake',
 'head_contains_fraud',
 'head_contains_hoax',
 'head_contains_false',
 'head_contains_deny',
 'head_contains_denies',
 'head_contains_refute',
 'head_contains_not',
 'head_contains_despite',
 'head_contains_nope',
 'head_contains_doubt',
 'head_contains_doubts',
 'head_contains_bogus',
 'head_contains_debunk',
 'head_contains_pranks',
 'head_contains_retract',
 'body_contains_fake',
 'body_contains_fraud',
 'body_contains_hoax',
 'body_contains_false',
 'body_contains_deny',
 'body_contains_denies',
 'body_contains_refute',
 'body_contains_not',
 'body_contains_despite',
 'body_contains_nope',
 'body_contains_doubt',
 'body_contains_doubts',
 'body_contains_bogus',
 'body_contains_debunk',
 'body_contains_pranks',
 'body_contains_retract']

In [11]:
well.input_cols = production_cols = [
 'head_2gram_unique_ratio',
 'head_3gram_unique_ratio',
 'head_4gram_unique_ratio',
 'body_2gram_unique_ratio',
 'body_3gram_unique_ratio',
 'body_4gram_unique_ratio'
] + ['head_contains_fake',
 'head_contains_fraud',
 'head_contains_hoax',
 'head_contains_false',
 'head_contains_deny',
 'head_contains_denies',
 'head_contains_refute',
 'head_contains_not',
 'head_contains_despite',
 'head_contains_nope',
 'head_contains_doubt',
 'head_contains_doubts',
 'head_contains_bogus',
 'head_contains_debunk',
 'head_contains_pranks',
 'head_contains_retract',
 'body_contains_fake',
 'body_contains_fraud',
 'body_contains_hoax',
 'body_contains_false',
 'body_contains_deny',
 'body_contains_denies',
 'body_contains_refute',
 'body_contains_not',
 'body_contains_despite',
 'body_contains_nope',
 'body_contains_doubt',
 'body_contains_doubts',
 'body_contains_bogus',
 'body_contains_debunk',
 'body_contains_pranks',
 'body_contains_retract'
]

In [25]:
well.y

array([[1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0]])

In [12]:
clf = DecisionTreeClassifier(well)
clf.fit()
clf.accuracy()

,f1,accuracy,recall,precision,accuracyImprovement
0,0.833333,0.714286,0.833333,0.833333,-0.035714
1,0.727273,0.571429,1.000000,0.571429,-0.178571
2,0.888889,0.833333,0.800000,1.000000,0.083333


In [13]:
clf = AdaBoosterClassifier(well, n_estimators=150)
clf.fit()
clf.accuracy()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,f1,accuracy,recall,precision,accuracyImprovement
0,0.833333,0.714286,0.833333,0.833333,-0.035714
1,0.727273,0.571429,1.000000,0.571429,-0.178571
2,0.800000,0.666667,0.800000,0.800000,-0.083333


In [14]:
clf = SVMClassifier(well)
clf.fit()
clf.accuracy()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,f1,accuracy,recall,precision,accuracyImprovement
0,0.923077,0.857143,1.0,0.857143,0.107143
1,0.727273,0.571429,1.0,0.571429,-0.178571
2,0.909091,0.833333,1.0,0.833333,0.083333


# Demonstrating BERT & Cayde

In [15]:
well.createBERTEncodings()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 20


INFO:tensorflow:Writing example 0 of 20


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 1 (id = 0)


INFO:tensorflow:label: 1 (id = 0)


['head_bert', 'body_bert']

In [16]:
well._df['head_bert']

2867    <bert.run_classifier.InputFeatures object at 0...
2094    <bert.run_classifier.InputFeatures object at 0...
2198    <bert.run_classifier.InputFeatures object at 0...
2520    <bert.run_classifier.InputFeatures object at 0...
1056    <bert.run_classifier.InputFeatures object at 0...
2478    <bert.run_classifier.InputFeatures object at 0...
3331    <bert.run_classifier.InputFeatures object at 0...
2       <bert.run_classifier.InputFeatures object at 0...
3079    <bert.run_classifier.InputFeatures object at 0...
1692    <bert.run_classifier.InputFeatures object at 0...
3376    <bert.run_classifier.InputFeatures object at 0...
1391    <bert.run_classifier.InputFeatures object at 0...
3946    <bert.run_classifier.InputFeatures object at 0...
3423    <bert.run_classifier.InputFeatures object at 0...
2479    <bert.run_classifier.InputFeatures object at 0...
2551    <bert.run_classifier.InputFeatures object at 0...
1387    <bert.run_classifier.InputFeatures object at 0...
4194    <bert.

In [17]:
dir(well._df['head_bert'][1758])

KeyError: 1758